In [1]:
import pandas as pd
import numpy as np

In [3]:
t_zacks_fc = pd.read_parquet('t_zacks_fc.parquet') 
t_zacks_fr = pd.read_parquet('t_zacks_fr.parquet') 
t_zacks_mktv = pd.read_parquet('t_zacks_mktv.parquet') 
t_zacks_shrs = pd.read_parquet('t_zacks_shrs.parquet')
t_zacks_sectors = pd.read_csv('t_zacks_sectors.csv')

t_zacks_mktv.tail()

,ticker,per_end_date,per_type,mkt_val
113694,LXRX,2024-09-30,Q,386.59
113695,LXU,2024-09-30,Q,575.83
113696,LVS,2024-09-30,Q,36497.82
113697,LWAY,2024-09-30,Q,383.38
113698,KLIC,2024-09-30,Q,2460.00


In [ ]:
'AAPL' in t_zacks_fc['ticker'].values

True

## Inner Merge

In [6]:
merged_df = t_zacks_fc.merge(t_zacks_fr, on=['ticker', 'per_end_date', 'per_type'], how='inner')
merged_df = merged_df.merge(t_zacks_mktv, on=['ticker', 'per_end_date', 'per_type'], how='inner')
merged_df = merged_df.merge(t_zacks_shrs, on=['ticker', 'per_end_date', 'per_type'], how='inner')
merged_df['zacks_sector_code'] = merged_df.groupby('ticker')['zacks_sector_code'].transform(lambda x: x.ffill().bfill())
merged_df

,ticker,comp_name,exchange,per_end_date,per_type,filing_date,filing_type,zacks_sector_code,lterm_debt_net_tot,eps_diluted_net_basic,ret_invst,tot_debt_tot_equity,mkt_val,shares_out
0,IDA,IDACORP,NYSE,2006-03-31,Q,2006-05-09,10-Q,14.0,-2.054,0.6000,1.3070,1.0541,1381.84,42.49
1,BMY,Bristol Myers Squibb,NYSE,2006-03-31,Q,2006-05-08,10-Q,4.0,2.000,0.3600,3.5999,0.7366,48325.68,1963.66
2,ABT,Abbott Laboratories,NYSE,2006-03-31,Q,2006-05-04,10-Q,4.0,-425.000,0.5600,4.5636,0.4357,65345.62,1538.63
3,AXGN,AxoGen,NASDAQ,2006-03-31,Q,2006-05-15,10-Q,4.0,NaN,-0.0100,-3.1985,NaN,2.41,4.15
4,JHG,Janus Henderson Group,NYSE,2006-03-31,Q,2006-04-27,10-Q,13.0,262.200,0.3602,1.2691,0.1492,4991.05,101.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97842,EXC,Exelon,NASDAQ,2024-09-30,Q,2024-10-30,10-Q,14.0,3639.000,0.7000,0.9998,1.7307,40746.00,1004.83
97843,NWE,NorthWestern,NASDAQ,2024-09-30,Q,2024-10-29,10-Q,14.0,115.000,0.7600,0.8691,1.0560,3508.40,61.31
97844,LNW,Light & Wonder,NASDAQ,2024-09-30,Q,2024-11-12,10-Q,2.0,-5.000,0.7100,1.3594,4.5140,8049.94,88.72
97845,IDA,IDACORP,NYSE,2024-09-30,Q,2024-10-31,10-Q,14.0,299.814,2.1200,1.7980,0.9530,5489.95,53.27


In [7]:
# 1. Earnings per Share (EPS)
merged_df['EPS'] = merged_df['eps_diluted_net_basic']

# 2. Return on Investments (ROI)
merged_df['ROI'] = merged_df['ret_invst']

# 3. Total Debt to Equity Ratio
merged_df['Debt_to_Equity_Ratio'] = merged_df['tot_debt_tot_equity']

# 4. Net Long-Term Debt
merged_df['Net_Long_Term_Debt'] = merged_df['lterm_debt_net_tot']

# 5. Market Capitalization
merged_df['Market_Capitalization'] = merged_df['mkt_val']

# 6. Price-to-Earnings Ratio (P/E)
merged_df['P_E_Ratio'] = merged_df['mkt_val'] / (merged_df['eps_diluted_net_basic'] * merged_df['shares_out'])
merged_df['P_E_Ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)

# 7. Market Cap to Shares Outstanding
merged_df['Market_Cap_per_Share'] = merged_df['mkt_val'] / merged_df['shares_out']

# 8. Earnings Growth Rate
merged_df = merged_df.sort_values(by=['ticker', 'per_end_date'])
merged_df['Previous_EPS'] = merged_df.groupby('ticker')['eps_diluted_net_basic'].shift(1)
merged_df['Earnings_Growth_Rate'] = (merged_df['eps_diluted_net_basic'] - merged_df['Previous_EPS']) / merged_df['Previous_EPS']

# 9. Debt to Market Cap Ratio
merged_df['Debt_to_Market_Cap_Ratio'] = merged_df['lterm_debt_net_tot'] / merged_df['mkt_val']

# 10. Sector Relative Valuation (P/E to Sector Average)
sector_pe_avg = merged_df.groupby('zacks_sector_code')['P_E_Ratio'].transform('mean')
merged_df['Sector_Relative_Valuation'] = merged_df['P_E_Ratio'] / sector_pe_avg

# 11. Relative Debt to Equity (compared with sector averages)
sector_de_avg = merged_df.groupby('zacks_sector_code')['tot_debt_tot_equity'].transform('mean')
merged_df['Relative_Debt_to_Equity'] = merged_df['tot_debt_tot_equity'] / sector_de_avg

# Display the updated DataFrame with new columns
risk_metrics_inner = merged_df[['ticker', 'comp_name', 'zacks_sector_code', 'per_end_date', 'EPS', 'ROI', 'Debt_to_Equity_Ratio', 'Net_Long_Term_Debt', 
                           'Market_Capitalization', 'P_E_Ratio', 'Market_Cap_per_Share', 
                           'Earnings_Growth_Rate', 'Debt_to_Market_Cap_Ratio', 
                           'Sector_Relative_Valuation', 'Relative_Debt_to_Equity']]

# Change sector code to real name
risk_metrics = risk_metrics_inner.merge(t_zacks_sectors, on='zacks_sector_code', how='left')
risk_metrics = risk_metrics.drop(columns=['zacks_sector_code'])
risk_metrics.head(10)

,ticker,comp_name,per_end_date,EPS,ROI,Debt_to_Equity_Ratio,Net_Long_Term_Debt,Market_Capitalization,P_E_Ratio,Market_Cap_per_Share,Earnings_Growth_Rate,Debt_to_Market_Cap_Ratio,Sector_Relative_Valuation,Relative_Debt_to_Equity,sector
0,AAON,AAON,2006-03-31,0.0395,4.4650,0.0017,-0.027,293.45,79.711523,3.148605,NaN,-0.000092,1.274818,0.001332,Construction
1,AAON,AAON,2006-06-30,0.0356,4.0712,0.0276,-0.054,315.77,94.916467,3.379026,-0.098734,-0.000171,1.517989,0.021631,Construction
2,AAON,AAON,2006-09-30,0.0553,6.0283,0.0735,-0.081,282.68,54.316800,3.003719,0.553371,-0.000287,0.868683,0.057604,Construction
3,AAON,AAON,2006-12-31,0.0474,4.9546,0.0006,-0.108,324.35,73.013519,3.460841,-0.142857,-0.000333,1.167697,0.000470,Construction
4,AAON,AAON,2007-03-31,0.0652,6.4285,0.0003,-0.027,322.31,52.774687,3.440910,0.375527,-0.000084,0.844020,0.000235,Construction
5,AAON,AAON,2007-06-30,0.0711,6.7000,0.0000,-0.054,396.56,58.989915,4.194183,0.090491,-0.000136,0.943419,0.000000,Construction
6,AAON,AAON,2007-09-30,0.0553,5.0375,NaN,-0.059,369.21,70.479171,3.897498,-0.222222,-0.000160,1.127166,NaN,Construction
7,AAON,AAON,2007-12-31,0.0494,4.7998,0.0010,0.271,372.02,79.254570,3.915176,-0.106691,0.000728,1.267510,0.000784,Construction
8,AAON,AAON,2008-03-31,0.0691,6.4192,0.0009,-0.023,361.76,57.260321,3.956688,0.398785,-0.000064,0.915758,0.000705,Construction
9,AAON,AAON,2008-06-30,0.0849,8.5934,0.0463,-0.045,346.24,44.810567,3.804417,0.228654,-0.000130,0.716651,0.036286,Construction


In [8]:
risk_metrics[risk_metrics['ticker'] == 'AAPL']

,ticker,comp_name,per_end_date,EPS,ROI,Debt_to_Equity_Ratio,Net_Long_Term_Debt,Market_Capitalization,P_E_Ratio,Market_Cap_per_Share,Earnings_Growth_Rate,Debt_to_Market_Cap_Ratio,Sector_Relative_Valuation,Relative_Debt_to_Equity,sector
150,AAPL,Apple,2006-03-31,0.0168,4.7224,NaN,NaN,53417.28,133.333333,2.240000,NaN,NaN,2.010581,NaN,Computer & Technology
151,AAPL,Apple,2006-06-30,0.0193,5.0589,NaN,NaN,48708.63,105.977063,2.045357,0.148810,NaN,1.598066,NaN,Computer & Technology
152,AAPL,Apple,2006-09-30,0.0218,5.4287,NaN,NaN,65663.06,126.114023,2.749286,0.129534,NaN,1.901719,NaN,Computer & Technology
153,AAPL,Apple,2006-12-31,0.0407,8.9419,NaN,NaN,72900.84,74.447184,3.030000,0.866972,NaN,1.122616,NaN,Computer & Technology
154,AAPL,Apple,2007-03-31,0.0311,6.2801,NaN,NaN,80076.88,106.695003,3.318215,-0.235872,NaN,1.608892,NaN,Computer & Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,AAPL,Apple,2023-09-30,1.4700,14.5820,1.7875,-5923.0,2662786.75,116.469406,171.210027,0.166667,-0.002224,1.756284,-15.538434,Computer & Technology
221,AAPL,Apple,2023-12-31,2.1800,20.0463,1.4580,95088.0,2973025.25,88.316517,192.530006,0.482993,0.031984,1.331757,-12.674147,Computer & Technology
222,AAPL,Apple,2024-03-31,1.5300,14.2364,1.4097,-3150.0,2629488.25,112.078440,171.480014,-0.298165,-0.001198,1.690071,-12.254283,Computer & Technology
223,AAPL,Apple,2024-06-30,1.4000,14.0271,1.5186,-7400.0,3202295.25,150.442823,210.619953,-0.084967,-0.002311,2.268582,-13.200932,Computer & Technology


In [9]:
# find the stocks in the database for each year
risk_metrics['per_end_date'] = pd.to_datetime(risk_metrics['per_end_date'])
risk_metrics['year'] = risk_metrics['per_end_date'].dt.year

risk_metrics.groupby('year')['ticker'].unique()

year
2006    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2007    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2008    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2009    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2010    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2011    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2012    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2013    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2014    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2015    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2016    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2017    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2018    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2019    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2020    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2021    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2022    [AAON, AAP, AAPL, AATC, ABCB, ABCP, ABEO, ABG,...
2023    [

In [11]:
# finding Apple's ticker
risk_metrics[risk_metrics['comp_name'].str.contains('apple', case=False, na=False)]['ticker'].unique()

array(['AAPL', 'MLP'], dtype=object)

## Creating risk metrics database

In [12]:
import sqlite3

# creating database and store data
conn = sqlite3.connect('Risk_Metrics_Database.db')
risk_metrics.to_sql('risk_metrics', conn, if_exists='replace', index=False)
print(f"Data successfully imported.")

# close database connection
conn.close()

Data successfully imported.
